# TensorFlow roBERTa with CNN Head
In this notebook, we tokenize the data, create question answer targets, and how to build a custom question answer head for roBERTa in TensorFlow. Note that HuggingFace transformers don't have a `TFRobertaForQuestionAnswering` so we must make our own from `TFRobertaModel`. Roberta with CNN head is used for Twitter Sentiment Extraction.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers==3.3.0

     |████████████████████████████████| 1.1MB 7.4MB/s 
     |████████████████████████████████| 1.2MB 44.3MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 3.0MB 52.5MB/s 


# Load Libraries, Data, Tokenizer
We will use HuggingFace transformers [here][1]

[1]: https://huggingface.co/transformers/

In [3]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import math
print('TF version',tf.__version__)

TF version 2.5.0


In [4]:
MAX_LEN = 310
PATH = '/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
SEED = 88888
LABEL_SMOOTHING = 0.1
tf.random.set_seed(SEED)
np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/Tweet Sentiment Extraction/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
test = pd.read_csv('/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/Tweet Sentiment Extraction/test.csv').fillna('')
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


# Training Data
We will now convert the training data into arrays that RoBERTa understands.

The tokenization logic below is inspired by Abhishek's PyTorch notebook [here][1].

[1]: https://www.kaggle.com/abhishek/roberta-inference-5-folds

In [6]:
ct = train.shape[0]
tt = test.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
input_ids_t = np.ones((tt,MAX_LEN), dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((tt,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((tt,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1

# Build RoBERTa Model
We use a pretrained roBERTa base model and add a custom question answer head. First tokens are input into `bert_model` and we use BERT's first output, i.e. `x[0]` below. These are embeddings of all input tokens and have shape `(batch_size, MAX_LEN, 768)`. Next we apply `tf.keras.layers.Conv1D(filters=1, kernel_size=1)` and transform the embeddings into shape `(batch_size, MAX_LEN, 1)`. We then flatten this and apply `softmax`, so our final output from `x1` has shape `(batch_size, MAX_LEN)`. These are one hot encodings of the start tokens indicies (for `selected_text`). And `x2` are the end tokens indicies.


In [7]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss


def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

# Metric

In [8]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# Train RoBERTa Model
We train with 5 Stratified KFolds (based on sentiment stratification). Each fold, the best model weights are saved and then reloaded before oof prediction and test prediction.

In [9]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=SEED) #originally 5 splits
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model, padded_model = build_model()
        
    #sv = tf.keras.callbacks.ModelCheckpoint(
    #    '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
    #    save_weights_only=True, mode='auto', save_freq='epoch')
    inpT = [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]]
    targetT = [start_tokens[idxT,], end_tokens[idxT,]]
    inpV = [input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]]
    targetV = [start_tokens[idxV,], end_tokens[idxV,]]
    # sort the validation data
    shuffleV = np.int32(sorted(range(len(inpV[0])), key=lambda k: (inpV[0][k] == PAD_ID).sum(), reverse=True))
    inpV = [arr[shuffleV] for arr in inpV]
    targetV = [arr[shuffleV] for arr in targetV]
    path = '/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/R CNN Weights/'
    weight_fn = path+'%s-roberta-%i.h5'%(VER,fold)
    for epoch in range(1, EPOCHS + 1):
        # sort and shuffle: We add random numbers to not have the same order in each epoch
        shuffleT = np.int32(sorted(range(len(inpT[0])), key=lambda k: (inpT[0][k] == PAD_ID).sum() + np.random.randint(-3, 3), reverse=True))
        # shuffle in batches, otherwise short batches will always come in the beginning of each epoch
        num_batches = math.ceil(len(shuffleT) / BATCH_SIZE)
        batch_inds = np.random.permutation(num_batches)
        shuffleT_ = []
        for batch_ind in batch_inds:
            shuffleT_.append(shuffleT[batch_ind * BATCH_SIZE: (batch_ind + 1) * BATCH_SIZE])
        shuffleT = np.concatenate(shuffleT_)
        # reorder the input data
        inpT = [arr[shuffleT] for arr in inpT]
        targetT = [arr[shuffleT] for arr in targetT]
        model.fit(inpT, targetT, 
            epochs=epoch, initial_epoch=epoch - 1, batch_size=BATCH_SIZE, verbose=DISPLAY, callbacks=[],
            validation_data=(inpV, targetV), shuffle=False)  # don't shuffle in `fit`
        save_weights(model, weight_fn)

    print('Loading model...')
    # model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    load_weights(model, weight_fn)

    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = padded_model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-2:b-1])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
687/687 [==============================] - 185s 209ms/step - loss: 2.8433 - activation_loss: 1.4287 - activation_1_loss: 1.4146 - val_loss: 2.5238 - val_activation_loss: 1.2809 - val_activation_1_loss: 1.2429
Epoch 2/2
687/687 [==============================] - 141s 205ms/step - loss: 2.5278 - activation_loss: 1.2787 - activation_1_loss: 1.2491 - val_loss: 2.5211 - val_activation_loss: 1.2903 - val_activation_1_loss: 1.2308
Epoch 3/3
687/687 [==============================] - 142s 207ms/step - loss: 2.4069 - activation_loss: 1.2239 - activation_1_loss: 1.1830 - val_loss: 2.5157 - val_activation_loss: 1.2738 - val_activation_1_loss: 1.2419
Loading model...
Predicting OOF...
172/172 [==============================] - 19s 97ms/step
Predicting Test...
111/111 [==============================] - 2s 14ms/step
>>>> FOLD 1 Jaccard = 0.708875080414647

#######

All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 159s 209ms/step - loss: 2.9007 - activation_loss: 1.4504 - activation_1_loss: 1.4504 - val_loss: 2.5340 - val_activation_loss: 1.2953 - val_activation_1_loss: 1.2388
Epoch 2/2
688/688 [==============================] - 142s 206ms/step - loss: 2.5594 - activation_loss: 1.2962 - activation_1_loss: 1.2631 - val_loss: 2.4818 - val_activation_loss: 1.2726 - val_activation_1_loss: 1.2092
Epoch 3/3
688/688 [==============================] - 151s 219ms/step - loss: 2.4748 - activation_loss: 1.2560 - activation_1_loss: 1.2188 - val_loss: 2.5087 - val_activation_loss: 1.2736 - val_activation_1_loss: 1.2351
Loading model...
Predicting OOF...
172/172 [==============================] - 19s 97ms/step
Predicting Test...
111/111 [==============================] - 2s 14ms/step
>>>> FOLD 2 Jaccard = 0.7094399282438114

#########################
### FOLD 3
#########################


...

688/688 [==============================] - 162s 214ms/step - loss: 2.9207 - activation_loss: 1.4740 - activation_1_loss: 1.4467 - val_loss: 2.5725 - val_activation_loss: 1.3069 - val_activation_1_loss: 1.2656
Epoch 2/2
688/688 [==============================] - 141s 205ms/step - loss: 2.5420 - activation_loss: 1.2887 - activation_1_loss: 1.2533 - val_loss: 2.5251 - val_activation_loss: 1.2732 - val_activation_1_loss: 1.2519
Epoch 3/3
688/688 [==============================] - 147s 213ms/step - loss: 2.4454 - activation_loss: 1.2434 - activation_1_loss: 1.2020 - val_loss: 2.5335 - val_activation_loss: 1.2628 - val_activation_1_loss: 1.2707
Loading model...
Predicting OOF...
172/172 [==============================] - 19s 95ms/step
Predicting Test...
111/111 [==============================] - 2s 14ms/step
>>>> FOLD 3 Jaccard = 0.703464335255475

#########################
### FOLD 4
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 159s 210ms/step - loss: 2.8848 - activation_loss: 1.4475 - activation_1_loss: 1.4372 - val_loss: 2.5801 - val_activation_loss: 1.3209 - val_activation_1_loss: 1.2591
Epoch 2/2
688/688 [==============================] - 143s 208ms/step - loss: 2.5475 - activation_loss: 1.2883 - activation_1_loss: 1.2592 - val_loss: 2.5940 - val_activation_loss: 1.3122 - val_activation_1_loss: 1.2817
Epoch 3/3
688/688 [==============================] - 159s 231ms/step - loss: 2.5061 - activation_loss: 1.2716 - activation_1_loss: 1.2344 - val_loss: 2.5422 - val_activation_loss: 1.2911 - val_activation_1_loss: 1.2511
Loading model...
Predicting OOF...
172/172 [==============================] - 19s 95ms/step
Predicting Test...
111/111 [==============================] - 2s 14ms/step
>>>> FOLD 4 Jaccard = 0.6992995439467965

#########################
### FOLD 5
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 172s 230ms/step - loss: 2.9478 - activation_loss: 1.4793 - activation_1_loss: 1.4685 - val_loss: 2.5443 - val_activation_loss: 1.2773 - val_activation_1_loss: 1.2670
Epoch 2/2
688/688 [==============================] - 146s 213ms/step - loss: 2.5595 - activation_loss: 1.2927 - activation_1_loss: 1.2668 - val_loss: 2.5170 - val_activation_loss: 1.2799 - val_activation_1_loss: 1.2371
Epoch 3/3
688/688 [==============================] - 151s 219ms/step - loss: 2.4670 - activation_loss: 1.2492 - activation_1_loss: 1.2178 - val_loss: 2.5170 - val_activation_loss: 1.2756 - val_activation_1_loss: 1.2414
Loading model...
Predicting OOF...
172/172 [==============================] - 19s 95ms/step
Predicting Test...
111/111 [==============================] - 2s 14ms/step
>>>> FOLD 5 Jaccard = 0.7014657855189478



In [10]:
print('>>>> OVERALL 5Fold CV Jaccard =',np.mean(jac))

>>>> OVERALL 5Fold CV Jaccard = 0.7045089346759356


In [11]:
print(jac) # Jaccard CVs

[0.708875080414647, 0.7094399282438114, 0.703464335255475, 0.6992995439467965, 0.7014657855189478]


# Test Data
We must tokenize the test data exactly the same as we tokenize the training data

In [12]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import math
print('TF version',tf.__version__)

TF version 2.5.0


In [13]:
MAX_LEN = 310
PAD_ID = 1
num_splits = 5
SEED = 88888

PATH = '/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)

In [14]:
test = pd.read_csv('/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/Tweet Sentiment Extraction/test.csv').fillna('')

tt = test.shape[0]
input_ids_t = np.ones((tt,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((tt,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((tt,MAX_LEN),dtype='int32')
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask_t[k,:len(enc.ids)+3] = 1

In [15]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss


def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

In [16]:
DISPLAY=1 # USE display=1 FOR INTERACTIVE
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

for fold in range(0,5):
  
  K.clear_session()
  model, padded_model = build_model()
  path = '/content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/R CNN Weights/'
  weight_fn = path+'v0-roberta-%i.h5'%(fold)

  print('Loading model...')
  # model.load_weights('%s-roberta-%i.h5'%(VER,fold))
  load_weights(model, weight_fn) 
  
  print('Predicting Test...')

  preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
  preds_start += preds[0]/num_splits
  preds_end += preds[1]/num_splits

All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
111/111 [==============================] - 13s 93ms/step


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
111/111 [==============================] - 13s 92ms/step


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
111/111 [==============================] - 13s 93ms/step


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
111/111 [==============================] - 13s 93ms/step


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at /content/gdrive/MyDrive/Internship Dataset/RoBERTa CNN/TF RoBERTa/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
111/111 [==============================] - 13s 93ms/step


In [17]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-2:b-1])
    all.append(st)

In [18]:
test['selected_text'] = all
test.to_csv('/content/gdrive/MyDrive/Internship Dataset/Real-Time Data/Analysis_Data.csv',index=False)